In [3]:
from typing import Sequence
from datasets.dataset_dict import DatasetDict
import fire
import datasets
import sys

sys.path.append("/home/meissner/shortcut-pruning")
import numpy as np
from transformers import AutoTokenizer
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from tqdm import tqdm
import torch 
from src.models.hf_model import SequenceClassificationTransformer
from src.models.hf_model_pruned import PruningTransformer

In [4]:
import sklearn as sk
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

In [5]:
mnli = datasets.load_dataset("glue", "mnli")

Reusing dataset glue (/home/meissner/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
negation_words = ["no", "not", "don't", "none", "nothing", 
                  "never", "aren’t", "isn’t", "weren’t", 
                  "neither", "don’t", "didn’t", "doesn’t", 
                  "cannot", "hasn’t", "won’t"]

In [7]:
neg_dataset = []
neg_dataset_labels = []
is_neg_cont = []
is_neg_ent = []

for sample in mnli['train']:
    hyp_bag = []
    # prem_bag = []
    for word in negation_words:
        if word in sample['hypothesis']:
            hyp_bag.append(1)
        else:
            hyp_bag.append(0)
    if any(hyp_bag) and sample['label'] == 2:
            is_neg_cont.append(True)
            is_neg_ent.append(False)
    elif any(hyp_bag) and sample['label'] == 0:
            is_neg_cont.append(False)
            is_neg_ent.append(True)
    else:
        is_neg_cont.append(False)
        is_neg_ent.append(False)
        
    neg_dataset.append(hyp_bag)
    neg_dataset_labels.append(sample['label'])

X = np.array(neg_dataset)
y = np.array(neg_dataset_labels)

In [8]:
X.shape, y.shape

((392702, 16), (392702,))

In [21]:
model = SGDClassifier(loss="log", max_iter=50000)
model.fit(X, y)

SGDClassifier(loss='log', max_iter=50000)

In [22]:
print(model.score(X[is_neg_cont], y[is_neg_cont]))
print(model.score(X[is_neg_ent], y[is_neg_ent]))

0.9994339951236503
0.0005697464628240433


In [23]:
model_outputs = model.predict_proba(X)

In [24]:
model_outputs

array([[0.3677667 , 0.35253326, 0.27970004],
       [0.3677667 , 0.35253326, 0.27970004],
       [0.3677667 , 0.35253326, 0.27970004],
       ...,
       [0.3677667 , 0.35253326, 0.27970004],
       [0.3677667 , 0.35253326, 0.27970004],
       [0.3677667 , 0.35253326, 0.27970004]])